In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer, pipeline
from sentence_transformers import CrossEncoder
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/foundations-of-science-book/pg39713.txt


In [2]:
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#####                                                     29.7%#######################                                     52.3%########################                                 57.3%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!ollama pull qwen3:1.7b > /dev/null 2>&1
!ollama pull qwen3-embedding:0.6b > /dev/null 2>&1

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIBGq1ksSYtGNMAtZSHUfaKGANP5qmCMjDALy7InCz0r/



time=2025-10-20T20:09:27.518Z level=INFO source=routes.go:1511 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GGML_VK_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_REMOTES:[olla

[GIN] 2025/10/20 - 20:09:27 | 200 |      78.731µs |       127.0.0.1 | HEAD     "/"


time=2025-10-20T20:09:27.963Z level=INFO source=types.go:112 msg="inference compute" id=GPU-1df0b5e8-ab5e-0f0b-4f4f-216233b7cc9e library=CUDA compute=6.0 name=CUDA0 description="Tesla P100-PCIE-16GB" libdirs=ollama,cuda_v12 driver=12.6 pci_id=00:04.0 type=discrete total="16.0 GiB" available="15.9 GiB"
time=2025-10-20T20:09:27.963Z level=INFO source=routes.go:1605 msg="entering low vram mode" "total vram"="16.0 GiB" threshold="20.0 GiB"
time=2025-10-20T20:09:29.465Z level=INFO source=download.go:177 msg="downloading 3d0b790534fe in 14 100 MB part(s)"
time=2025-10-20T20:09:35.076Z level=INFO source=download.go:177 msg="downloading ae370d884f10 in 1 1.7 KB part(s)"
time=2025-10-20T20:09:36.703Z level=INFO source=download.go:177 msg="downloading d18a5cc71b84 in 1 11 KB part(s)"
time=2025-10-20T20:09:41.448Z level=INFO source=download.go:177 msg="downloading cff3f395ef37 in 1 120 B part(s)"
time=2025-10-20T20:09:43.042Z level=INFO source=download.go:177 msg="downloading 517ccaff02fe in 1 48

[GIN] 2025/10/20 - 20:09:48 | 200 |  20.42740552s |       127.0.0.1 | POST     "/api/pull"
[GIN] 2025/10/20 - 20:09:48 | 200 |      26.047µs |       127.0.0.1 | HEAD     "/"


time=2025-10-20T20:09:49.749Z level=INFO source=download.go:177 msg="downloading 06507c7b4268 in 7 100 MB part(s)"
time=2025-10-20T20:09:56.357Z level=INFO source=download.go:177 msg="downloading 9202febed9e2 in 1 266 B part(s)"


[GIN] 2025/10/20 - 20:09:59 | 200 | 10.974847071s |       127.0.0.1 | POST     "/api/pull"


In [4]:
!ollama list

[GIN] 2025/10/20 - 20:09:59 | 200 |      32.579µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/10/20 - 20:09:59 | 200 |     830.822µs |       127.0.0.1 | GET      "/api/tags"
NAME                    ID              SIZE      MODIFIED               
qwen3-embedding:0.6b    ac6da0dfba84    639 MB    Less than a second ago    
qwen3:1.7b              8f68893c685c    1.4 GB    11 seconds ago            


# RAG with Qwen 3 and FAISS

## Load model and embedder

In [5]:
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings

llm = ChatOllama(model="qwen3:1.7b")
embedder = OllamaEmbeddings(model="qwen3-embedding:0.6b")

In [14]:
import torch
reranker = CrossEncoder("tomaarsen/Qwen3-Reranker-0.6B-seq-cls")

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

## Load documents into FAISS

In [7]:
loader = TextLoader(
    file_path="/kaggle/input/foundations-of-science-book/pg39713.txt"
)
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [9]:
vector_store = FAISS.from_documents(all_splits, embedder)

time=2025-10-20T20:11:24.983Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat
time=2025-10-20T20:11:24.983Z level=WARN source=types.go:753 msg="invalid option provided" option=tfs_z
time=2025-10-20T20:11:24.983Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat_eta
time=2025-10-20T20:11:24.983Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat_tau
time=2025-10-20T20:11:25.302Z level=WARN source=server.go:209 msg="flash attention enabled but not supported by model"
time=2025-10-20T20:11:25.302Z level=INFO source=server.go:400 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model /root/.ollama/models/blobs/sha256-06507c7b42688469c4e7298b0a1e16deff06caf291cf0a5b278c308249c3e439 --port 39905"
time=2025-10-20T20:11:25.303Z level=INFO source=server.go:676 msg="loading model" "model layers"=29 requested=-1
time=2025-10-20T20:11:25.303Z level=INFO source=server.go:682 msg="system memory

[GIN] 2025/10/20 - 20:12:44 | 200 |         1m19s |       127.0.0.1 | POST     "/api/embed"


## Create prompt

In [10]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Create QA chain

In [ ]:
retriever = vector_store.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

### Get results

In [ ]:
query = "Can you explain thermodynamics?"
result = qa_chain({"query": query})

print(result["result"])

In [ ]:
query = "Who is the author The Foundations of Science?"
result = qa_chain({"query": query})

print(result["result"])

## QA chain with reranking

In [27]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from typing import List
from pydantic import PrivateAttr

DEFAULT_TASK = (
    "Given a question about the book, rank the passages by how useful they are for answering the question. "
    "Only consider content from the book that directly supports a correct and concise answer."
)

def rerank_documents(query, docs, task=DEFAULT_TASK, top_k=3):
    def format_query(q):
        return (
            f"<|im_start|>system\n"
            f"Judge whether the Document meets the requirements based on the Query and the Instruct provided. "
            f"Note that the answer can only be \"yes\" or \"no\"."
            f"<|im_end|>\n<|im_start|>user\n"
            f"<Instruct>: {task}\n<Query>: {q}\n"
        )

    def format_doc(d):
        return f"<Document>: {d}<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"

    texts = [d.page_content if isinstance(d, Document) else d for d in docs]
    pairs = [(format_query(query), format_doc(t)) for t in texts]
    scores = reranker.predict(pairs)
    ranked_idx = np.argsort(scores)[::-1]
    return [docs[i] for i in ranked_idx[:top_k]]

class RerankingRetriever(BaseRetriever):
    # use these as private attributes instead of the ones of the base class
    _base_retriever: BaseRetriever = PrivateAttr()
    _top_k: int = PrivateAttr()

    def __init__(self, base_retriever, top_k=3):
        super().__init__()
        self._base_retriever = base_retriever
        self._top_k = top_k

    def _get_relevant_documents(self, query: str) -> List[Document]:
        docs = self._base_retriever.get_relevant_documents(query)
        return rerank_documents(query, docs, top_k=self._top_k)

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        docs = await self._base_retriever.aget_relevant_documents(query)
        return rerank_documents(query, docs, top_k=self._top_k)

reranking_retriever = RerankingRetriever(retriever, top_k=3)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=reranking_retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

### Get results

In [28]:
query = "Who is the author The Foundations of Science?"
result = qa_chain({"query": query})

print(result["result"])

time=2025-10-20T20:43:59.219Z level=WARN source=types.go:753 msg="invalid option provided" option=tfs_z
time=2025-10-20T20:43:59.219Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat
time=2025-10-20T20:43:59.219Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat_eta
time=2025-10-20T20:43:59.219Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat_tau
time=2025-10-20T20:43:59.531Z level=WARN source=server.go:209 msg="flash attention enabled but not supported by model"
time=2025-10-20T20:43:59.531Z level=INFO source=server.go:400 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model /root/.ollama/models/blobs/sha256-06507c7b42688469c4e7298b0a1e16deff06caf291cf0a5b278c308249c3e439 --port 35309"
time=2025-10-20T20:43:59.532Z level=INFO source=server.go:676 msg="loading model" "model layers"=29 requested=-1
time=2025-10-20T20:43:59.532Z level=INFO source=server.go:682 msg="system memory

[GIN] 2025/10/20 - 20:44:00 | 200 |  1.316194281s |       127.0.0.1 | POST     "/api/embed"


time=2025-10-20T20:44:00.398Z level=INFO source=server.go:1310 msg="llama runner started in 0.87 seconds"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ggml_backend_cuda_device_get_memory utilizing NVML memory reporting free: 9021030400 total: 17179869184
time=2025-10-20T20:44:01.948Z level=INFO source=sched.go:545 msg="updated VRAM based on existing loaded models" gpu=GPU-1df0b5e8-ab5e-0f0b-4f4f-216233b7cc9e library=CUDA total="16.0 GiB" available="8.4 GiB"
time=2025-10-20T20:44:02.029Z level=INFO source=server.go:216 msg="enabling flash attention"
time=2025-10-20T20:44:02.029Z level=INFO source=server.go:400 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model /root/.ollama/models/blobs/sha256-3d0b790534fe4b79525fc3692950408dca41171676ed7e21db57af5c65ef6ab6 --port 45011"
time=2025-10-20T20:44:02.030Z level=INFO source=server.go:676 msg="loading model" "model layers"=29 requested=-1
time=2025-10-20T20:44:02.030Z level=INFO source=server.go:682 msg="system memory" total="31.4 GiB" free="25.2 GiB" free_swap="0 B"
time=2025-10-20T20:44:02.030Z level=INFO source=server.go:690 msg="gpu memory" id=GPU-1df0b5e8-ab

The author of *The Foundations of Science* is Henri Poincaré. Josiah Royce wrote the introduction, but the main author of the book is Poincaré. The work includes translations of "Science and Hypothesis," "The Value of Science," and "Science and Method."
[GIN] 2025/10/20 - 20:44:05 | 200 |  3.624498158s |       127.0.0.1 | POST     "/api/chat"


In [30]:
# To check if reranking is working as expected

base_docs = retriever.get_relevant_documents("Who is the author The Foundations of Science?")
print("=== Top 10 from FAISS ===")
for i, doc in enumerate(base_docs):
    print(f"{i+1}:", doc.page_content[:200], "...")  # preview first 200 chars

# Then rerank them using your reranker
shortlist_docs = rerank_documents("Some question about the book", base_docs, top_k=3)
print("\n=== Shortlist passed to Ollama ===")
for i, doc in enumerate(shortlist_docs):
    print(f"{i+1}:", doc.page_content[:200], "...")

[GIN] 2025/10/20 - 20:47:59 | 200 |  152.733637ms |       127.0.0.1 | POST     "/api/embed"
=== Top 10 from FAISS ===
1: Author: Henri Poincaré

Author of introduction, etc.: Josiah Royce

Translator: George Bruce Halsted

Release date: May 16, 2012 [eBook #39713]

Language: English

Credits: Produced by Bryan Ness and  ...
2: ﻿The Project Gutenberg eBook of The Foundations of Science: Science and Hypothesis, The Value of Science, Science and Method
    
This ebook is for the use of anyone anywhere in the United States and
 ...
3: Volume III. University Control. By J. MCKEEN CATTELL and other
      authors.

    AMERICAN MEN OF SCIENCE. A Biographical Directory.

    SCIENCE. A weekly journal devoted to the advancement of scien ...
4: A SERIES OF VOLUMES FOR THE PROMOTION OF
  SCIENTIFIC RESEARCH AND EDUCATIONAL PROGRESS

  EDITED BY J. MCKEEN CATTELL


  VOLUME I--THE FOUNDATIONS OF SCIENCE




  UNDER THE SAME EDITORSHIP


    SC ...
5: Rados, "permit me to make especial mention of h

time=2025-10-20T20:47:59.141Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat
time=2025-10-20T20:47:59.141Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat_tau
time=2025-10-20T20:47:59.141Z level=WARN source=types.go:753 msg="invalid option provided" option=mirostat_eta
time=2025-10-20T20:47:59.141Z level=WARN source=types.go:753 msg="invalid option provided" option=tfs_z


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


=== Shortlist passed to Ollama ===
1: Author: Henri Poincaré

Author of introduction, etc.: Josiah Royce

Translator: George Bruce Halsted

Release date: May 16, 2012 [eBook #39713]

Language: English

Credits: Produced by Bryan Ness and  ...
2: THE VALUE OF SCIENCE

    Translator's Introduction                              201
    Does the Scientist Create Science?                     201
    The Mind Dispelling Optical Illusions            ...
3: Every age has ridiculed the one before it, and accused it of having
generalized too quickly and too naïvely. Descartes pitied the Ionians;
Descartes, in his turn, makes us smile. No doubt our children ...


ggml_backend_cuda_device_get_memory utilizing NVML memory reporting free: 6976307200 total: 17179869184
ggml_backend_cuda_device_get_memory utilizing NVML memory reporting free: 9018933248 total: 17179869184
